In [1]:
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

In [2]:
#Read initial data
file_path='rtfDataSet.csv'
df = pd.read_csv(file_path)
print(df.head())

       0      1      2      3      4      5      6      7      8      9  ...  \
0 -0.536  0.292 -0.684  0.123 -0.118  0.346 -0.308 -0.113  0.401 -0.399  ...   
1 -0.496 -0.298 -0.529  0.073 -0.072  0.107 -0.316 -0.066 -0.168 -0.099  ...   
2  0.008 -0.031 -0.240  0.178 -0.120  0.317 -0.481  0.031 -0.077  0.063  ...   
3 -0.188 -0.180 -0.062 -0.104 -0.136 -0.061 -0.216 -0.143  0.068 -0.189  ...   
4  0.038  0.155 -0.203 -0.088 -0.084 -0.164 -0.145 -0.168  0.008  0.143  ...   

      61     62     63     64     65     66     67     68     69  label  
0 -0.135 -0.063 -0.410 -0.223 -0.599 -0.136 -0.329 -0.132 -0.266      0  
1  0.189  0.007 -0.362 -0.151 -0.338 -0.031 -0.159 -0.097 -0.131      0  
2 -0.035  0.101 -0.098 -0.172 -0.290  0.033 -0.342 -0.321 -0.238      0  
3  0.146 -0.145 -0.527 -0.292 -0.663 -0.078 -0.194 -0.151 -0.268      0  
4  0.019 -0.084  0.164 -0.165 -0.126 -0.112  0.029 -0.138  0.148      1  

[5 rows x 71 columns]


In [3]:
#Reduce to sets and normalized
set_2=[2, 5, 9, 10, 16, 21, 28, 34, 38, 48, 49, 55, 61, 62, 64]
set_3=[1, 2, 7, 8, 9, 10, 19, 23, 28, 34, 35, 36, 42, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 63, 64, 65, 67]

reduced_df_2=df.iloc[:,set_2]
reduced_df_3=df.iloc[:,set_3]

y = df.iloc[:, -1]
X_amigo = df.iloc[:, :-1]

scaler = MinMaxScaler()

reduced_df_2 = scaler.fit_transform(reduced_df_2)
reduced_df_3 = scaler.fit_transform(reduced_df_3)

#Array of sets
reduced_array=[reduced_df_2,reduced_df_3]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_amigo, y, test_size=0.2, random_state=42)

In [62]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters

# Define the search space for hyperparameters
hp = HyperParameters()
hp.Choice('learning_rate', values=[0.1, 0.3, 0.5])
hp.Int('neurons', min_value=5, max_value=20, step=5)
hp.Choice('activation', values=['relu', 'sigmoid', 'linear'])
hp.Int('num_layers', min_value=1, max_value=5)  # Hyperparameter for the number of layers

# Define your input shape based on your data
input_shape = (X_train.shape[1],)

# Create a function to build the Keras model
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))

    for _ in range(hp.get('num_layers')):
        model.add(keras.layers.Dense(
            units=hp.Int('neurons', min_value=5, max_value=20, step=5),
            activation=hp.Choice('activation', ['relu', 'sigmoid', 'linear']), 
            kernel_regularizer=keras.regularizers.l1_l2(
                l1=hp.Choice('l1', [0.001, 0.0001, 0.005, 0.0045]),  # L1 regularization
                l2=hp.Choice('l2', [0.001, 0.0001, 0.005, 0.0045])  # L2 regularization
        )))
      

    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.005, 0.1, 0.3, 0.5])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the tuner (Random Search in this case)
tuner = RandomSearch(
    build_model, objective='val_accuracy', max_trials=70, hyperparameters=hp)

# Perform the hyperparameter search
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

# Print the best hyperparameters
print("Best Hyperparameters:", best_hyperparameters)

# Train the best model with the entire training dataset
best_model.fit(X_train, y_train, epochs=10, batch_size=32)

Trial 70 Complete [00h 00m 03s]
val_accuracy: 0.699999988079071

Best val_accuracy So Far: 0.8125
Total elapsed time: 00h 03m 21s
Best Hyperparameters: <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x00000187275D5A00>
Epoch 1/10
10/10 [==============================] - 1s 2ms/step - loss: 0.4945 - accuracy: 0.8766
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 0.4754 - accuracy: 0.8987
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: 0.5210 - accuracy: 0.8481
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.5605 - accuracy: 0.8544
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 0.5824 - accuracy: 0.8259
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 0.4938 - accuracy: 0.8797
Epoch 7/10
10/10 [==============================] - 0s 3ms/step - loss: 0.5293 - accuracy: 0.8766
Epoch 8/10
10/10 [==============================] - 0s 3ms/s

In [63]:
print("Best Hyperparameters:", best_hyperparameters.values)

Best Hyperparameters: {'learning_rate': 0.1, 'neurons': 10, 'activation': 'relu', 'num_layers': 3, 'l1': 0.001, 'l2': 0.001}


### Nuevo Gridsearch

In [14]:
mlp = MLPClassifier()

# Define the parameter grid to search
param_grid = {
    'hidden_layer_sizes': [(5), (5,5), (5,5,5), (5,5,5,5),(5,5,5,5,5),
                           (10), (10,10), (10,10,10), (10,10,10,10),(10,10,10,10,10),
                           (15), (15,15), (15,15,15), (15,15,15,15),(15,15,15,15,15),
                           (20), (20,20), (20,20,20), (20,20,20,20),(20,20,20,20,20),
                           (25), (25,25), (25,25,25), (25,25,25,25),(25,25,25,25,25)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.001, 0.0001, 0.005, 0.0045,0.002,0.0025],
    'learning_rate_init': [0.1,0.3,0.5],
    'batch_size':[32,64]
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Test Accuracy:", accuracy)

KeyboardInterrupt: 

## Model 1

In [14]:
#Creacion dinamica de folds y clasificacion y metricas
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter=1
epochs = [1,50,100,150,200,250,300,350,400,450,500]

#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):
        for epoch in epochs:
            print("epoch parametr: ",epoch)
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]
            mlp = MLPClassifier(hidden_layer_sizes=(15,15,15), activation='relu', max_iter=epoch, random_state=42,learning_rate_init=0.1,alpha=0.001,batch_size=32, verbose=False,n_iter_no_change=500)
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)

            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            print(f"Fold {fold + 1}: Accuracy on the validation set: {accuracy_val * 100:.2f}%")


Set:  1
epoch parametr:  1
Fold 1: Accuracy on the validation set: 50.00%
epoch parametr:  50


c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 1: Accuracy on the validation set: 50.00%
epoch parametr:  100


c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 1: Accuracy on the validation set: 50.00%
epoch parametr:  150


...

c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 10: Accuracy on the validation set: 48.72%
epoch parametr:  450


c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (450) reached and the optimization hasn't converged yet.
  warnings.warn(


Fold 10: Accuracy on the validation set: 51.28%
epoch parametr:  500
Fold 10: Accuracy on the validation set: 48.72%


c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            mlp = MLPClassifier(hidden_layer_sizes=(32,), activation='relu', max_iter=100, random_state=42)
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(6, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.5)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))
            

## Model 2

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.3)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.5)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

## Model 3